In [1]:
# libraries
import os
import sys
import json
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../extra_files'))
import helper as hp
from imageio import imwrite, imread
from skimage.transform import resize

In [2]:
# File paths
data_path = '/home/aldo/Documents/data-cic/'
preprocess_path = data_path + 'preprocess_data'

## Training SSD300 trained with mobilenet backbone trained

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from light_models.keras_ssd300_squeezenetv1_bypass import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files.f1_callback import F1_callback as f1_call

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


### Parameters (original SSD300 architecture)

In [4]:
## Parameteres needed for ssd_300() and SSDInputEncoder()

img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [1., 1., 1.] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
divide_by_stddev = [127.5, 127.5, 127.5]
swap_channels = False # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales = scales_pascal
#scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [16, 30, 60, 100, 150, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

## Create new model with SSD weights

In [5]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                divide_by_stddev=divide_by_stddev,
                swap_channels=swap_channels)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

adam = Adam(lr=0.001)
#sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
identity_layer (Lambda)         (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
input_stddev_normalization (Lam (None, 300, 300, 3)  0           identity_layer[0][0]             
__________________________________________________________________________________________________
input_mean_normalization (Lambd (None, 300, 300, 3)  0           input_stddev_normalization[0][0] 
__________________________________________________________________________________________________
model_2 (M

## Data generator for the training

In [7]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
images_dir = data_path + 'images_reduced'

# Ground truth
#train_labels_filename = preprocess_path + '/cic_train_reduced.csv'
train_labels_filename = preprocess_path + '/cic_train_val_reduced.csv'
val_labels_filename   = preprocess_path + '/cic_val_reduced.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 45/45 [00:00<00:00, 164.29it/s]
Number of images in the training dataset:	   255
Number of images in the validation dataset:	    45


In [8]:
# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv9_mbox_conf').output_shape[1:3],
                   model.get_layer('conv10_mbox_conf').output_shape[1:3],
                   model.get_layer('conv11_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv12_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv13_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv14_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   255
Number of images in the validation dataset:	    45


## Remaining training parameters

In [9]:
# Define a learning rate schedule.
def lr_schedule(epoch):
    if epoch < 400:
        return 0.001
    else:
        return 0.0001

In [10]:
K.clear_session() # Clear previous models from memory.
model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                divide_by_stddev=divide_by_stddev,
                swap_channels=swap_channels)

adam = Adam(lr=0.001)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.load_weights('/home/aldo/Documents/weights/light_models/PASCAL/squeezenet_v1_bypass.h5')
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
# Define model callbacks.
main_path = '/home/aldo/Documents/'
# TODO: Set the filepath under which you want to save the model.

csv_logger = CSVLogger(filename=main_path + 'data-cic/history/light_models/CIC/squeezenet_v1_bypass.csv',
                           separator=',',
                           append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule, verbose=1)


callbacks = [csv_logger,
             learning_rate_scheduler]
    
initial_epoch   = 0
final_epoch     = 500
steps_per_epoch = 15

history = model.fit_generator(generator=train_generator,
                                  steps_per_epoch=steps_per_epoch,
                                  epochs=final_epoch,
                                  callbacks=callbacks,
                                  initial_epoch=initial_epoch)
    
model.save(main_path + 'weights/light_models/CIC/squeezenet_v1_bypass.h5')

Epoch 1/500

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 12s 804ms/step - loss: 5.2841
Epoch 2/500

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 726ms/step - loss: 4.5006
Epoch 3/500

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 745ms/step - loss: 4.4172
Epoch 4/500

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 739ms/step - loss: 4.3756
Epoch 5/500

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 736ms/step - loss: 4.3046
Epoch 6/500

Epoch 00006: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 752ms/step - loss: 4.4487
Epoch 7/500

Epoch 00007: LearningRateScheduler setting learning rate to 0.001.
15/15 [=======

Epoch 55/500

Epoch 00055: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 681ms/step - loss: 3.3705
Epoch 56/500

Epoch 00056: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 700ms/step - loss: 3.3524
Epoch 57/500

Epoch 00057: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 696ms/step - loss: 3.1705
Epoch 58/500

Epoch 00058: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 679ms/step - loss: 3.2036
Epoch 59/500

Epoch 00059: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 735ms/step - loss: 3.2552
Epoch 60/500

Epoch 00060: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 705ms/step - loss: 3.3257
Epoch 61/500

Epoch 00061: LearningRateScheduler setting learning rate to 0.001.
15/15 [

15/15 [==============================] - 11s 706ms/step - loss: 2.9202
Epoch 109/500

Epoch 00109: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 681ms/step - loss: 2.9409
Epoch 110/500

Epoch 00110: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 680ms/step - loss: 2.9843
Epoch 111/500

Epoch 00111: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 697ms/step - loss: 3.0846
Epoch 112/500

Epoch 00112: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 701ms/step - loss: 2.9770
Epoch 113/500

Epoch 00113: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 700ms/step - loss: 2.9375
Epoch 114/500

Epoch 00114: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 680ms/step - loss: 2.8578
Epoch 115/5

15/15 [==============================] - 10s 676ms/step - loss: 2.8233
Epoch 163/500

Epoch 00163: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 707ms/step - loss: 2.7434
Epoch 164/500

Epoch 00164: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 695ms/step - loss: 2.7069
Epoch 165/500

Epoch 00165: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 695ms/step - loss: 2.6780
Epoch 166/500

Epoch 00166: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 692ms/step - loss: 2.7681
Epoch 167/500

Epoch 00167: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 703ms/step - loss: 2.8110
Epoch 168/500

Epoch 00168: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 710ms/step - loss: 2.7098
Epoch 169/5

15/15 [==============================] - 10s 698ms/step - loss: 2.6668
Epoch 217/500

Epoch 00217: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 697ms/step - loss: 2.6241
Epoch 218/500

Epoch 00218: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 678ms/step - loss: 2.6185
Epoch 219/500

Epoch 00219: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 704ms/step - loss: 2.5254
Epoch 220/500

Epoch 00220: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 698ms/step - loss: 2.5157
Epoch 221/500

Epoch 00221: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 675ms/step - loss: 2.4915
Epoch 222/500

Epoch 00222: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 685ms/step - loss: 2.5645
Epoch 223/5

15/15 [==============================] - 10s 694ms/step - loss: 2.4179
Epoch 271/500

Epoch 00271: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 699ms/step - loss: 2.3953
Epoch 272/500

Epoch 00272: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 681ms/step - loss: 2.4760
Epoch 273/500

Epoch 00273: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 708ms/step - loss: 2.4602
Epoch 274/500

Epoch 00274: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 717ms/step - loss: 2.4728
Epoch 275/500

Epoch 00275: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 674ms/step - loss: 2.5253
Epoch 276/500

Epoch 00276: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 686ms/step - loss: 2.4572
Epoch 277/5

15/15 [==============================] - 11s 706ms/step - loss: 2.4085
Epoch 325/500

Epoch 00325: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 684ms/step - loss: 2.3249
Epoch 326/500

Epoch 00326: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 728ms/step - loss: 2.4734
Epoch 327/500

Epoch 00327: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 696ms/step - loss: 2.2902
Epoch 328/500

Epoch 00328: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 664ms/step - loss: 2.4587
Epoch 329/500

Epoch 00329: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 691ms/step - loss: 2.3971
Epoch 330/500

Epoch 00330: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 681ms/step - loss: 2.4119
Epoch 331/5

15/15 [==============================] - 11s 728ms/step - loss: 2.5222
Epoch 379/500

Epoch 00379: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 669ms/step - loss: 2.2503
Epoch 380/500

Epoch 00380: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 698ms/step - loss: 2.3758
Epoch 381/500

Epoch 00381: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 679ms/step - loss: 2.2943
Epoch 382/500

Epoch 00382: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 10s 693ms/step - loss: 2.2461
Epoch 383/500

Epoch 00383: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 702ms/step - loss: 2.2133
Epoch 384/500

Epoch 00384: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 11s 700ms/step - loss: 2.3283
Epoch 385/5

15/15 [==============================] - 10s 692ms/step - loss: 1.9424
Epoch 432/500

Epoch 00432: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 11s 710ms/step - loss: 2.0271
Epoch 433/500

Epoch 00433: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 10s 695ms/step - loss: 1.9351
Epoch 434/500

Epoch 00434: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 11s 710ms/step - loss: 1.9691
Epoch 435/500

Epoch 00435: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 10s 667ms/step - loss: 1.8750
Epoch 436/500

Epoch 00436: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 10s 676ms/step - loss: 1.8786
Epoch 437/500

Epoch 00437: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 10s 686ms/step - loss: 1.8761
Epoch

15/15 [==============================] - 11s 752ms/step - loss: 1.8554
Epoch 485/500

Epoch 00485: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 12s 785ms/step - loss: 1.8155
Epoch 486/500

Epoch 00486: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 10s 691ms/step - loss: 1.6942
Epoch 487/500

Epoch 00487: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 12s 823ms/step - loss: 1.8269
Epoch 488/500

Epoch 00488: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 11s 759ms/step - loss: 1.8292
Epoch 489/500

Epoch 00489: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 11s 738ms/step - loss: 1.9022
Epoch 490/500

Epoch 00490: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 11s 734ms/step - loss: 1.7554
Epoch